In [1]:
import sys
sys.path.append('./')
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Dense, Flatten
from keras.layers import Convolution1D
from keras.layers import MaxPooling1D
from keras.layers import Embedding
from keras.layers import ThresholdedReLU
from keras.layers import Dropout
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras import layers
from keras import models
import h5py
from keras.models import Model
from keras import Input
from keras import optimizers
import os, shutil
from os import listdir
from os.path import isfile, join, isdir

Using TensorFlow backend.
/home/caio/aibox/env-flask-nlp/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/caio/aibox/env-flask-nlp/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/caio/aibox/env-flask-nlp/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/caio/aibox/env-flask-nlp/l

In [2]:
classes = {}
images_path = 'data/tobacco3482jpg/'
classes_dirs = [f for f in listdir(images_path) if isdir(join(images_path, f))]
print('Classes Found:')
for image_class in classes_dirs:
    classes[image_class] = {}
    print(image_class)

Classes Found:
Note
News
Scientific
Memo
Email
Form
ADVE
Resume
Letter
Report


In [5]:
base_dir = 'data/tobacco3482jpg/'


train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

for image_class in classes_dirs:
    
    classes[image_class]['train_dir'] = os.path.join(train_dir, image_class)
    os.mkdir(classes[image_class]['train_dir'])

    classes[image_class]['validation_dir'] = os.path.join(validation_dir, image_class)
    os.mkdir(classes[image_class]['validation_dir'])



In [6]:
for image_class in classes_dirs:
    images_path = 'data/tobacco3482jpg/{}/'.format(image_class)
    classes[image_class]['images'] = [f for f in listdir(images_path) if isfile(join(images_path, f))]
    

In [8]:
for image_class, class_data in classes.items():
    print(image_class)
    print(len(class_data['images']))
    print(class_data['images'])

Note
202
['88025179.jpg', '2022245046.jpg', '2030195129.jpg', '2030053173.jpg', '80191212_80191213.jpg', '2029183905.jpg', '93443778.jpg', '2030353547.jpg', '50076168.jpg', '2063603124.jpg', '2028837643.jpg', '1000240946.jpg', '03023805.jpg', '2072821881.jpg', '2020102257.jpg', '2065023135.jpg', '2023104578.jpg', '2067474399.jpg', '2085234526.jpg', '2023192377.jpg', '91985231-a_91985231.jpg', '2029042370.jpg', '2072489111.jpg', '2048150028.jpg', '2070159121.jpg', '2072571651.jpg', '1005127604_1005127605.jpg', '2028826508.jpg', '2044183733.jpg', '2029026171.jpg', '00093561.jpg', '2025325000.jpg', '2045518260.jpg', '2063710424.jpg', '01404111.jpg', '91771720.jpg', '2062055279.jpg', '0000628923.jpg', '2085052974.jpg', '2077122151.jpg', '82857902.jpg', '2076140545.jpg', '2074784438.jpg', '2051800478.jpg', '10226038.jpg', '2031367558.jpg', '2029204890.jpg', '03563613.jpg', '1003188772.jpg', '10106776.jpg', '2065354661.jpg', '2071226076.jpg', '87927212.jpg', '2060570691.jpg', '1005111374.jpg

In [9]:
train_size = 0.7
validation_size = 1 - train_size

for image_classe, class_data in classes.items():
    class_data['len_train'] = int(len(class_data['images']) * train_size)
    class_data['len_validation'] = int(len(class_data['images']) * validation_size)

In [12]:
for image_class, class_data in classes.items():
    original_dir = base_dir + image_class + '/'
    images = class_data['images']
    len_train = class_data['len_train']
    for file in images[:len_train]:        
        src = os.path.join(original_dir, file)
        dst = os.path.join(class_data['train_dir'], file)
        shutil.copyfile(src, dst)
    
    for file in images[len_train:]:
        src = os.path.join(original_dir, file)
        dst = os.path.join(class_data['validation_dir'], file)
        shutil.copyfile(src, dst)



In [15]:
for image_class, class_data in classes.items():
    print('\ntotal training {} images:'.format(image_class), len(os.listdir(class_data['train_dir'])))
    print('total validation {} images:'.format(image_class), len(os.listdir(class_data['validation_dir'])))


total training Note images: 141
total validation Note images: 61

total training News images: 132
total validation News images: 57

total training Scientific images: 183
total validation Scientific images: 79

total training Memo images: 434
total validation Memo images: 187

total training Email images: 420
total validation Email images: 180

total training Form images: 302
total validation Form images: 130

total training ADVE images: 161
total validation ADVE images: 70

total training Resume images: 84
total validation Resume images: 37

total training Letter images: 397
total validation Letter images: 171

total training Report images: 186
total validation Report images: 80


In [16]:
image_height = 150
image_width = 150

In [24]:
def nll1(y_true, y_pred):
    """ Negative log likelihood. """

    # keras.losses.binary_crossentropy give the mean
    # over the last axis. we require the sum
    return K.sum(K.binary_crossentropy(y_true, y_pred), axis=-1)

# CNN

In [26]:
image_input = Input(shape=(image_height, image_width, 1), name='sent_input', dtype='float32')
x = layers.Conv2D(20, (7, 7), activation='relu',input_shape=(image_height, image_width, 1))(image_input)
x = layers.MaxPooling2D((4, 4))(x)
x = layers.Conv2D(50, (5, 5), activation='relu')(x)
x = layers.MaxPooling2D((4, 4))(x)
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
output = layers.Dense(10, activation='softmax')(x)

model_func = Model(image_input, output)

In [27]:
sgd = optimizers.SGD(lr=0.01)

model_func.compile(loss=nll1,
optimizer=sgd,
metrics=['acc'])

In [28]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(image_height, image_width),
    batch_size=20,
    class_mode='binary',
    subset='training',
    color_mode='grayscale')

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(image_height, image_width),
    batch_size=20,
    class_mode='binary',
    subset='validation',
    color_mode='grayscale')

print(len(train_generator))
print(len(validation_generator))


Found 5574 images belonging to 12 classes.
Found 1390 images belonging to 12 classes.
279
70


In [29]:
history = model_func.fit_generator(
train_generator,
steps_per_epoch=train_generator.samples,
epochs=8,
validation_data=validation_generator,
validation_steps=validation_generator.samples,
verbose=1)


Epoch 1/8
  45/5574 [..............................] - ETA: 32:46 - loss: 153.1228 - acc: 0.1300

KeyboardInterrupt: 

In [ ]:
model_func.save('document_classification.h5')